# FARLAB - Robotable Streets Project 
Developer: @mattwfranchi

Project Members: Matt Franchi, Maria-Teresa Parreira, Frank Bu, Wendy Ju 

As robots deployments become more common, they will become yet another dancer in the sidewalk ballet. Within urban mapping, transit mobility and walkability scores have emerged as a way to measure the quality of a city's infrastructure for a specific medium of traffic. However, there is no such metric for robots! Here, we aim to envision what a 'robotability' score might look like, and how it might be used to inform urban planning and policy. 

We utilize the following data features in computing a *robotability score*: 
- Sidewalk width 
- Sidewalk quality proxied by 311 complaints 
- Pedestrian density, computed via aggregated dashcam data
- Sidewalk material (concrete, asphalt, cobblestone, etc.) 
- Connectivity: cellular coverage, WiFi availbility, IoT network coverage, and GPS coverage 
- Elevation change from beginning to end of road segment 
- Solar radiation levels, for potential solar charging and for potential overheating. 
- Proximity to hypothetical charging stations 
- Grating on sidewalk (ie in NYC, the subway grates) that might be problematic for robots to navigate 
- Snow buildup 
- Local attitudes towards robots 
- Average illegal parking levels, ie cars parked on sidewalks 
- Shade / shadows 
- Overhead covering (scaffolding, awnings, etc., in the case of non-waterproof bots)
- Zoning. My hypothesis: robots are more acceptable in commercial-zoned areas, and less acceptable in majority-residential zoned areas. 

### Other Things to Lock In (4/25/24): 
- Study period. Some of this data (311 complaints, pedestrian densities, etc., should be constrained within a time range. **Limitation: we don't have new, free dashcam data presently**)



In [19]:
# class RobotabilityGraph that inherits from Graph class 
import sys 
sys.path.append("/share/ju/urban-fingerprinting")

import osmnx as ox 
import geopandas as gpd 
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 

from glob import glob 
from tqdm import tqdm 

from shapely import wkt, LineString 

from src.utils.logger import setup_logger 

logger = setup_logger('robotability-score')
logger.setLevel("INFO")
logger.info("Modules initialized.")

WGS='EPSG:4326'
PROJ='EPSG:2263'

REGEN_SEGMENTIZATION=False


2024-08-15 18:42:12 - robotability-score - INFO - Modules initialized.


## Loading and Preprocessing Data Features 

### Neighborhood Tabulation Areas (NYC)

In [20]:
# Load the Neighborhood Tabulation Areas (NTAs) dataset 
ntas_nyc = pd.read_csv("data/ntas_nyc.csv")
ntas_nyc = gpd.GeoDataFrame(ntas_nyc, geometry=wkt.loads(ntas_nyc['the_geom']), crs=WGS).to_crs(PROJ)

In [21]:
# Remove redundant columns 
TO_DROP = ['BoroCode','CountyFIPS','NTA2020','NTAAbbrev','CDTA2020','CDTAName']
ntas_nyc = ntas_nyc.drop(columns=TO_DROP)

logger.success("NTAs loaded.")

2024-08-15 18:42:12 - robotability-score - SUCCESS - NTAs loaded.


### Sidewalk Basemap (NYC)

In [22]:
# Load the NYC sidewalk basemap 
sidewalk_nyc = pd.read_csv("data/sidewalks_nyc.csv")
sidewalk_nyc = gpd.GeoDataFrame(sidewalk_nyc, geometry=wkt.loads(sidewalk_nyc['the_geom']), crs=WGS).to_crs(PROJ)

In [23]:
# Take out features we don't need, and add a width column 
TO_DROP = ['SUB_CODE', 'FEAT_CODE', 'STATUS', 'the_geom']
sidewalk_nyc = sidewalk_nyc.drop(columns=TO_DROP)
sidewalk_nyc['SHAPE_Width'] = sidewalk_nyc['SHAPE_Area'] / sidewalk_nyc['SHAPE_Leng']

# Simplify 
sidewalk_nyc['geometry'] = sidewalk_nyc['geometry'].simplify(10)

# write to disk 
if REGEN_SEGMENTIZATION:
    # segmentize 
    segmentized = sidewalk_nyc.segmentize(50).extract_unique_points().explode(index_parts=True)

    segmentized = gpd.GeoDataFrame(segmentized).reset_index() 

    segmentized = segmentized.merge(sidewalk_nyc,left_on='level_0',right_index=True).drop(columns=['level_0','level_1','geometry'])
    segmentized['geometry'] = segmentized.iloc[:,0]
    segmentized.drop(segmentized.columns[0],axis=1, inplace=True)
    segmentized = gpd.GeoDataFrame(segmentized, crs=PROJ)

    segmentized.to_file("data/sidewalks_nyc_segmentized.geojson", driver='GeoJSON')
    logger.success("Segmentized sidewalk basemap written to disk.")

else: 
    segmentized = gpd.read_file("data/sidewalks_nyc_segmentized.geojson")
    logger.info("Segmentized sidewalk basemap loaded.")


sidewalk_nyc = segmentized

logger.success("NYC sidewalk basemap loaded.")
logger.info(f"Distribution of sidewalk widths [ft]: \n{sidewalk_nyc['SHAPE_Width'].describe()}")

2024-08-15 18:44:37 - robotability-score - INFO - Segmentized sidewalk basemap loaded.
2024-08-15 18:44:37 - robotability-score - SUCCESS - NYC sidewalk basemap loaded.
2024-08-15 18:44:37 - robotability-score - INFO - Distribution of sidewalk widths [ft]: 
count    2.551208e+06
mean     5.373540e+00
std      1.480766e+00
min      2.710948e-01
25%      4.458703e+00
50%      5.149090e+00
75%      6.177234e+00
max      4.021491e+01
Name: SHAPE_Width, dtype: float64
